# Sistema de Compressão

$ \frac{d\dot{m}}{dt} = \frac{A_1}{L_C}(\phi (N(t), \dot{m})P_1 - P_P(t)) $

$ \frac{d P_P}{dt} = \frac{C_1^2}{\nu _P}(\dot{m}(t) - \alpha (t) K_\nu \sqrt{P_P - P_{out}}) $

$ \begin{matrix} A_1 & = & 2.6\centerdot 10^-3 m² \\
\nu _P & = & 2.0 m³ \\
L_C & = & 2.0 m \\
K_\nu & = & \frac{0.38 kg}{(kPa)^{0.5}s} \\
P_1 & = & 4.5 Bar \\
P{out} & = & 5.0 Bar \end{matrix}
$

$ \frac{d\dot{m}}{dt} = \frac{2.6\centerdot 10^{-3}}{2.0}(1.5\centerdot 4.5 - P_P) $

$ \frac{d P_P}{dt} = \frac{479.029^2}{2.0}(\dot{m} - \alpha {0.38} \sqrt{P_P - 5.0}) $

#### Importações

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.optimize import fsolve
import casadi as ca
import plotly.graph_objects as go
from plotly.subplots import make_subplots

2024-09-30 14:33:20.576544: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 14:33:20.584846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 14:33:20.594898: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 14:33:20.597734: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 14:33:20.605316: I tensorflow/core/platform/cpu_feature_guar

#### Constantes e Variáveis Auxiliares

In [2]:
np.random.seed(42)

# Constantes
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38
P1 = 4.5
P_out = 5
C = 479

timestep = 2 # Passos no passado para prever o próximo
epochs = 50000
units = 100 # Número de Neurônios
nData = 60 # Segundos para cada alteração do Alfa
nAlphas = 10 # Número de vezes que o Alfa irá mudar, considere o treino e os testes.
alphas = np.random.uniform(0.35,0.65, nAlphas) # Abertura da válvula

# Variáveis auxiliares
interval = [np.linspace(i * nData, (i + 1) * nData, nData) for i in range(nAlphas)]
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []
RNN_test = []
nAlphas_Met = int(nAlphas/2)

### Solução Numérica

##### Cálculo da Solução

In [3]:
# Condições Iniciais
def fun(variables, A1, Lc, kv, P1, P_out, C) :
    (x,y) = variables
    eqn_1 = (A1/Lc)* ((1.5 * P1) - y)
    eqn_2 = (C**2)/2 * (x - alphas[0] * kv * np.sqrt(y - P_out))
    return [eqn_1, eqn_2]

result = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C)) 

init_m = result[0] 
init_p = result[1]

# Variáveis CasADi
x = ca.MX.sym('x', 2)
alpha = ca.MX.sym('alpha', 1)

# Solução Numérica
for i in range(0,nAlphas): 
    alpha_values.append([np.full(nData, alphas[i])])

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1]), (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1] - P_out)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }

    F = ca.integrator('F','idas', ode, interval[i][0], interval[i])
    
    sol = F(x0 = [init_m, init_p], p = alphas[i])

    xf_values = np.array(sol["xf"])

    aux1, aux2 = xf_values
    massFlowrate.append(aux1)
    PlenumPressure.append(aux2)
    init_m = aux1[-1]
    init_p = aux2[-1]

    if i < nAlphas_Met:
        RNN_train.append([aux1, aux2, np.full(nData,alphas[i])])
        RNN_trainFut.append([aux1, aux2, np.full(nData,alphas[i+1])])
    else:
        RNN_test.append([aux1, aux2, np.full(nData,alphas[i])])


##### Gráfico do Modelo

In [4]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Vazão vs Tempo", "Pressão vs Tempo", "Alpha vs Tempo"))

for i in range(0, nAlphas):
    # Vazão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',
                             name='Vazão', legendgroup='massflow', showlegend=i == 0), row = 1, col = 1)
    # Pressão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines',
                             name='Pressão', legendgroup='pressure', showlegend=i == 0), row = 1, col = 2)
    # Alphas
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(alpha_values[i]), mode='lines', 
                             name='Alphas', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 1, col = 3)

# Atualiza layout
fig.update_layout(
    xaxis_title='Tempo',
    grid=dict(rows=1, columns=3),
    template='plotly',
    showlegend=False
)

# Mostra a figura
fig.show()


### Rede Neural

##### Tratamento de Dados

In [5]:
RNN_train = np.array(RNN_train)
RNN_trainFut = np.array(RNN_trainFut)
RNN_test = np.array(RNN_test)

X_train = []
y_train = []
x_test = []

# Nao estou gostando de ter que separar o RNN Train e RNN Test

for j in range(nAlphas_Met):
    for i in range(len(RNN_train[0][0])):
        X_train.append(RNN_train[j,:,i])
        x_test.append(RNN_test[j,:,i])
        if i == (len(RNN_train[0][0]) - 1):
            y_train.append(RNN_trainFut[j, :2, 0])
        else:
            y_train.append(RNN_train[j,:2,i+1])

newData = int((nData*nAlphas_Met)/timestep)

X_train = np.array(X_train)
y_train = np.array(y_train)
x_test = np.array(x_test)

X_train = tf.reshape(X_train, [newData,timestep,3])
y_train = tf.reshape(y_train, [newData,timestep,2])
x_test = tf.reshape(x_test, [newData,timestep,3])

I0000 00:00:1727717601.759756 1197777 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727717601.783407 1197777 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727717601.783533 1197777 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727717601.784450 1197777 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

##### Rede e Treinamento

In [6]:
model = keras.Sequential()
model.add(layers.Bidirectional(layers.SimpleRNN(units, input_shape=(timestep,3), return_sequences= True, activation='tanh')))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(2))
model.add(layers.Lambda(lambda x: x[:, -1:, :]))

def lossCustom(y_true, y_pred):
    weight1 = 1e3
    weight2 = 1
    data_loss = tf.reduce_mean(tf.square(y_true[:, :, 0] - y_pred[:, :, 0]))* weight1 + tf.reduce_mean(tf.square(y_true[:, :, 1] - y_pred[:, :, 1]))*weight2
    return data_loss


model.compile(optimizer = 'adam', loss = lossCustom)
early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss',  
    patience=units*10,     
    restore_best_weights=True 
)

history = model.fit(X_train, y_train, epochs=epochs, verbose=0, callbacks= [early_stopping])

/home/guilhermefreire/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

I0000 00:00:1727717602.699088 1197879 service.cc:146] XLA service 0x7688500043d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727717602.699107 1197879 service.cc:154]   StreamExecutor device (0): NVIDIA T400 4GB, Compute Capability 7.5
2024-09-30 14:33:22.725903: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-30 14:33:22.831619: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-09-30 14:33:24.585594: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:762] The NVIDIA driver's CUDA version is 12.2 

##### Gráfico Rede Neural

In [7]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Loss","Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

# Loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)), y=history.history['loss'], mode='lines', name='Mass Loss', line=dict(dash='solid')),
                  row=1, col=1)

prediction = np.array(model.predict(x_test))
mass = prediction[:,:,0]
pressure = prediction[:, :,1]

interval2 = [np.linspace(newData*timestep, newData*timestep*2, newData) for i in range(nAlphas)]

for i in range(nAlphas_Met, nAlphas):
    for j in range(0, len(interval2[i]), nData):
        end = j + nData
        if end > len(interval2[i]):
            end = len(interval2[i])
        
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(mass[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(pressure[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)
    # Modelo
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

fig.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig.show()

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
